# Quantum Fisher Information

Orjan Ameye  
2026-01-26

This notebook demonstrates the computation of Quantum Fisher Information
(QFI) for a driven-dissipative Kerr Parametric Oscillator (KPO) using
automatic differentiation. The QFI quantifies the ultimate precision
limit for parameter estimation in quantum systems.

We import the necessary packages for quantum simulations and automatic
differentiation:

In [1]:
using QuantumToolbox   # Quantum optics simulations
using ForwardDiff      # For automatic differentiation
using LinearAlgebra    # Linear algebra operations
using CairoMakie       # Plotting
CairoMakie.activate!(type = "svg")

## System Parameters and Hamiltonian

The KPO system is governed by the Hamiltonian:
$$H = -p_1 a^\dagger a + K (a^\dagger)^2 a^2 - G (a^\dagger a^\dagger + a a)$$

where:

-   $p_1$ is the parameter we want to estimate (detuning)
-   $K$ is the Kerr nonlinearity
-   $G$ is the parametric drive strength
-   $\gamma$ is the decay rate

In [1]:
const G, K, γ = 0.002, 0.001, 0.01

const N = 20 # cutoff of the Hilbert space dimension
const a = destroy(N) # annihilation operator

const c_ops = [sqrt(γ)*a]
const ψ0 = fock(N, 0) # initial state

const tlist = range(0, 2000, 100)

function final_state(p, t)
    H = - p[1] * a' * a + K * a' * a' * a * a - G * (a' * a' + a * a)

    sol = mesolve(H, ψ0, tlist, c_ops; params = p, progress_bar = Val(false), saveat = [t])
    ρ_vec = vec(sol.states[end].data)

    # Split the real and imaginary parts of the density matrix to a real vector since ForwardDiff doesn't handle complex numbers directly
    return vcat(real.(ρ_vec), imag.(ρ_vec))
end

final_state (generic function with 1 method)

## Quantum Fisher Information Calculation

The QFI is computed using the symmetric logarithmic derivative (SLD).
For a density matrix $\rho(\theta)$ parametrized by $\theta$:

$$F_Q = \text{Tr}[\partial_\theta \rho \cdot L]$$

where $L$ is the SLD satisfying:
$\partial_\theta \rho = \frac{1}{2}(\rho L + L \rho)$

In [1]:
function compute_fisher_information(ρ, dρ)
    # Add small regularization to avoid numerical issues with zero eigenvalues
    reg = 1e-12 * I
    ρ_reg = ρ + reg

    # Solve for the symmetric logarithmic derivative L
    # dρ = (1/2)(ρL + Lρ)
    # This is a Sylvester equation: ρL + Lρ = 2*dρ
    L = sylvester(ρ_reg, ρ_reg, -2*dρ)

    # Fisher information F = Tr(dρ * L)
    F = real(tr(dρ * L))

    return F
end

compute_fisher_information (generic function with 1 method)

## Automatic Differentiation Setup

We use finite differences through DifferentiationInterface.jl to compute
the derivative of the quantum state with respect to the parameter. This
is a key step that enables efficient QFI computation without manual
derivative calculations.

In [1]:
# Test the system
final_state([0], 100)

# Define state function for automatic differentiation
state(p) = final_state(p, 2000)

# Compute both the state and its derivative
ρ, dρ = state([0.0]), ForwardDiff.jacobian(state, [0.0])

# Reshape back to complex matrix form
ρ = reshape(ρ[1:end÷2] + 1im * ρ[end÷2+1:end], N, N)

# Reshape the derivative back to matrix form
dρ = reshape(dρ[1:end÷2] + 1im * dρ[end÷2+1:end], N, N)

# Compute QFI at final time
qfi_final = compute_fisher_information(ρ, dρ)
println("QFI at final time: ", qfi_final)

QFI at final time: 19795.72214787345

## Time Evolution of Quantum Fisher Information

Now we compute how the QFI evolves over time to understand the optimal
measurement time for parameter estimation:

In [1]:
ts = range(0, 2000, 100)

@time QFI_t = map(ts) do t
    state(p) = final_state(p, t)
    ρ, dρ = state([0.0]), ForwardDiff.jacobian(state, [0.0])

    ρ = reshape(ρ[1:end÷2] + 1im * ρ[end÷2+1:end], N, N)
    dρ = reshape(dρ[1:end÷2] + 1im * dρ[end÷2+1:end], N, N)

    compute_fisher_information(ρ, dρ)
end

println("QFI computed for ", length(ts), " time points")

 10.660851 seconds (18.54 M allocations: 1.029 GiB, 1.61% gc time, 55.84% compilation time)
QFI computed for 100 time points

## Visualization

Plot the time evolution of the Quantum Fisher Information:

In [1]:
fig = Figure()
ax = Axis(
 fig[1,1],
 xlabel = "Time",
 ylabel = "Quantum Fisher Information"
)
lines!(ax, ts, QFI_t)
fig

## Version Information

In [1]:
QuantumToolbox.versioninfo()


 QuantumToolbox.jl: Quantum Toolbox in Julia
≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡
Copyright © QuTiP team 2022 and later.
Current admin team:
    Alberto Mercurio and Yi-Te Huang

Package information:
Julia              Ver. 1.12.4
QuantumToolbox     Ver. 0.41.0
SciMLOperators     Ver. 1.14.1
LinearSolve        Ver. 3.57.0
OrdinaryDiffEqCore Ver. 2.3.0

System information:
OS       : Linux (x86_64-linux-gnu)
CPU      : 4 × AMD EPYC 7763 64-Core Processor
Memory   : 15.616 GB
WORD_SIZE: 64
LIBM     : libopenlibm
LLVM     : libLLVM-18.1.7 (ORCJIT, znver3)
BLAS     : libopenblas64_.so (ilp64)
Threads  : 4 (on 4 virtual cores)

+---------------------------------------------------+
| Please cite QuantumToolbox.jl in your publication |
+---------------------------------------------------+
For your convenience, a bibtex reference can be easily generated using `QuantumToolbox.cite()`.


In [1]:
using Pkg
Pkg.status()